In [1]:
import numpy as np
import json

In [21]:
def write_group(f, members, name):
    result = {'name': name, 'size': len(members), 'members': []}
    for m in members:
        i = np.linalg.inv(m)
        result['members'].append({'g': list(m.flatten()), 'i': list(i.flatten())})
    json.dump(result, f, indent=True)
    
def cgroup_2d(n):
    thetas = np.linspace(0, 2 * np.pi, n)
    r = []
    for t in thetas:
        s, c = np.sin(t), np.cos(t)
        r.append(np.array([[c, -s], [s, c]]))
    return r
def sgroup_2d(n):
    thetas = np.arange(0, n) / n
    r = [np.eye(2)]
    for t in thetas:
        s, c = np.sin(2 * np.pi * t), np.cos(2 * np.pi * t)
        r.append(np.array([[c, s], [s, -c]]))
    return r

def outer(g1, g2, remove_dups = True):
    r = []
    for i in g1:
        for j in g2:            
            add = True
            if remove_dups:                
                for ri in r:
                    if np.allclose(i @ j, ri):
                        add = False
                        break            
            if add:
                r.append(i @ j)
    return r

In [27]:
c8 = cgroup_2d(8)
s4 = sgroup_2d(4)
with open('group.json', 'w') as f:
    write_group(f, c8, 'C8')

In [28]:
print(c8)

[array([[ 1., -0.],
       [ 0.,  1.]]), array([[ 0.6234898 , -0.78183148],
       [ 0.78183148,  0.6234898 ]]), array([[-0.22252093, -0.97492791],
       [ 0.97492791, -0.22252093]]), array([[-0.90096887, -0.43388374],
       [ 0.43388374, -0.90096887]]), array([[-0.90096887,  0.43388374],
       [-0.43388374, -0.90096887]]), array([[-0.22252093,  0.97492791],
       [-0.97492791, -0.22252093]]), array([[ 0.6234898 ,  0.78183148],
       [-0.78183148,  0.6234898 ]]), array([[ 1.0000000e+00,  2.4492936e-16],
       [-2.4492936e-16,  1.0000000e+00]])]
